In [1]:
import wandb
import math
import random
import torch, torchvision
import torch.nn as nn
import torchvision.transforms as T
import pickle
import pandas as pd
import geopandas as gpd

import gnn_io

from my_gnn import MyGNN

In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: enatterer (tum-traffic-engineering). Use `wandb login --relogin` to force relogin


True

In [3]:
# Load the dictionary
with open('../results/result_dic.pkl', 'rb') as f:
    results_dict = pickle.load(f)

datasets = []
for key, df in results_dict.items():
    print(f"Policy: {key}")
    
    if isinstance(df, pd.DataFrame):
        gdf = gpd.GeoDataFrame(df, geometry='geometry')
        gdf.crs = "EPSG:2154"  # Assuming the original CRS is EPSG:2154
        gdf.to_crs("EPSG:4326", inplace=True)
        edge_index, car_volume_tensor, policy_tensor, nodes = gnn_io.create_edge_index_and_tensors(gdf)
        datasets.append((policy_tensor, car_volume_tensor))
    else:
        print(f"The value for key '{key}' is not a GeoDataFrame.")

Policy: policy introduced in Arrondissement(s) 5, 6
Policy: policy introduced in Arrondissement(s) 1, 2, 3
Policy: policy introduced in Arrondissement(s) 3, 4
Policy: policy introduced in Arrondissement(s) 16, 17, 18
Policy: policy introduced in Arrondissement(s) 13, 14, 15
Policy: policy introduced in Arrondissement(s) 8, 9, 10, 11
Policy: policy introduced in Arrondissement(s) 12, 13
Policy: policy introduced in Arrondissement(s) 2, 3, 4, 5, 6, 7
Policy: policy introduced in Arrondissement(s) 9, 10, 11
Policy: policy introduced in Arrondissement(s) 15, 16, 17, 18, 19
Policy: policy introduced in Arrondissement(s) 7, 8, 9, 10, 11, 12
Policy: policy introduced in Arrondissement(s) 13, 14
Policy: policy introduced in Arrondissement(s) 2, 3, 4, 5, 6
Policy: policy introduced in Arrondissement(s) 5
Policy: policy introduced in Arrondissement(s) 2
Policy: policy introduced in Arrondissement(s) 11, 12, 13, 14
Policy: policy introduced in Arrondissement(s) 12, 13, 14, 15
Policy: policy intro

In [4]:
edge_index.shape

torch.Size([2, 31216])

In [7]:
gnn = MyGNN(policy_input_dim=3, traffic_input_dim=1, hidden_dim=8)

In [ ]:
asdfasf

In [35]:
policy_input_dim = 3  # Dimensionality of policy features: capacity, freespeed flow, modes
traffic_input_dim = 1  # Dimensionality of traffic flow features
hidden_dim = 32  # Dimensionality of hidden representations
num_nodes = max(max(edge_index[0]), max(edge_index[1])) + 1  # Number of nodes in the graph

device = "cuda:0" if torch.cuda.is_available() else "cpu"

def get_dataloader(is_train, batch_size, slice=5):
    "Get a training dataloader"
    # full_dataset = torchvision.datasets.MNIST(root=".", train=is_train, transform=T.ToTensor(), download=True)
    # sub_dataset = torch.utils.data.Subset(full_dataset, indices=range(0, len(full_dataset), slice))
    loader = torch.utils.data.DataLoader(dataset=datasets, 
                                         batch_size=batch_size, 
                                         shuffle=True if is_train else False, 
                                         pin_memory=True, num_workers=2)
    return loader


def normalize(tensor):
    mean = tensor.mean(dim=0, keepdim=True)
    std = tensor.std(dim=0, keepdim=True) + 1e-6  # Add a small epsilon to avoid division by zero
    return (tensor - mean) / std

def validate_model(model, valid_dl, loss_func):
    "Compute performance of the model on the validation dataset and log a wandb.Table"
    model.eval()
    val_loss = 0
    correct = 0
    total = 0 
    with torch.inference_mode():
        correct = 0
        total = 0  # Add a total counter for accuracy calculation

        for i, (policy_features, flow_targets) in enumerate(valid_dl):
             # Normalize data and labels
            policy_features = normalize(policy_features.float())  # Shape: [105, 31216, 3]
            flow_targets = normalize(flow_targets.float().unsqueeze(2))  # Shape: [105, 31216, 1]

            # Forward pass ➡
            outputs = gnn(edge_index, policy_features, flow_targets)

            # Compute loss
            loss = loss_func(outputs, flow_targets)
            print(f"Loss: {loss.item()}")
            val_loss += loss.item() * flow_targets.size(0)  # Sum up the batch loss scaled by the number of examples

            # Compute accuracy and accumulate
            _, predicted = torch.max(outputs.data, 1)
            total += flow_targets.size(0)  # Increment total by the number of labels
            print("Flow targets shape: "+ str(flow_targets.size(0)))
            correct += (predicted == flow_targets.squeeze()).sum().item()  # Squeeze flow_targets to match predicted shape


            # val_loss += loss_func(outputs, flow_targets).item() * flow_targets
            
            # # Compute accuracy and accumulate
            # _, predicted = torch.max(outputs.data, 1)
            # total += flow_targets.size(0)  # Increment total by the number of labels
            # correct += (predicted == flow_targets.size(0)).sum().item()
            
    # return val_loss / len(valid_dl.dataset), correct / len(valid_dl.dataset)
    return val_loss / total, correct / total  # Average loss over all examples, accuracy as correct predictions over total


In [36]:
# Launch 5 experiments, trying different dropout rates

learning_rates = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]  # List of learning rates to iterate over

for lr in learning_rates:
    # 🐝 initialise a wandb run
    wandb.init(
        project="my_project_diff_learning_rates",
        config={
            "epochs": 10,
            "batch_size": 21,
            "lr": lr
    })
    
    # Copy your config 
    config = wandb.config

    # Get the data
    train_dl = get_dataloader(is_train=True, batch_size=config.batch_size)
    valid_dl = get_dataloader(is_train=False, batch_size=2*config.batch_size)
    n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)
    
    # Make the loss and optimizer
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(gnn.parameters(), lr=config.lr)

   # Training
    example_ct = 0
    step_ct = 0
    for epoch in range(config.epochs):
        print("epoch", epoch)
        gnn.train()
        for step, (policy_features, flow_targets) in enumerate(train_dl):            
            data, labels = normalize(policy_features.float()), normalize(flow_targets.float().unsqueeze(2))        
            outputs = gnn(edge_index, data, labels)
            train_loss = loss_func(outputs, labels)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            example_ct += len(data)
            metrics = {"train/train_loss": train_loss, 
                       "train/epoch": (step + 1 + (n_steps_per_epoch * epoch)) / n_steps_per_epoch, 
                       "train/example_ct": example_ct}
            if step + 1 < n_steps_per_epoch:
                # 🐝 Log train metrics to wandb 
                wandb.log(metrics)
                
            step_ct += 1

        val_loss, accuracy = validate_model(gnn, valid_dl, loss_func)

        # 🐝 Log train and validation metrics to wandb
        val_metrics = {"val/val_loss": val_loss, 
                       "val/val_accuracy": accuracy}
        wandb.log({**metrics, **val_metrics})
        print("Train loss", train_loss)
        print("Valid loss", val_loss)
        print("Accuracy", accuracy)
        # print(f"Train Loss: {train_loss:.3f}, Valid Loss: {val_loss:3f}, Accuracy: {accuracy:.2f}")

    # If you had a test set, this is how you could log it as a Summary metric
    wandb.summary['test_accuracy'] = 0.8

    # 🐝 Close your wandb run 
    wandb.finish()

epoch 0
Loss: -16221.8515625
Loss: -17120.380859375
Loss: -17906.666015625
Train loss tensor(-16425.2617, grad_fn=<DivBackward1>)
Valid loss -16918.226171875
Accuracy 0.0
epoch 1
Loss: -16227.1611328125
Loss: -17126.28515625
Loss: -17912.80859375
Train loss tensor(-16447.4375, grad_fn=<DivBackward1>)
Valid loss -16923.940234375
Accuracy 0.0
epoch 2
Loss: -16232.630859375
Loss: -17132.19140625
Loss: -17918.953125
Train loss tensor(-17948.2383, grad_fn=<DivBackward1>)
Valid loss -16929.71953125
Accuracy 0.0
epoch 3
Loss: -16238.3037109375
Loss: -17137.5234375
Loss: -17924.71484375
Train loss tensor(-17598.5723, grad_fn=<DivBackward1>)
Valid loss -16935.273828125
Accuracy 0.0
epoch 4
Loss: -16243.6728515625
Loss: -17143.80859375
Loss: -17931.0
Train loss tensor(-18256.5469, grad_fn=<DivBackward1>)
Valid loss -16941.192578125
Accuracy 0.0
epoch 5
Loss: -16249.2734375
Loss: -17149.5234375
Loss: -17937.046875
Train loss tensor(-16592.5723, grad_fn=<DivBackward1>)
Valid loss -16946.928125
Acc

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/train_loss,▃▄▅▆█▄▄▃▅▂▇▅█▅▄▅▄▅▄▇▅▆▄▂▃▅▆▆▆▅▆▁▇▃▃▃▅▄▄▃
val/val_accuracy,▁▁▁▁▁▁▁▁▁▁
val/val_loss,█▇▆▆▅▄▃▃▂▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,1050
train/train_loss,-17412.26172
val/val_accuracy,0.0
val/val_loss,-16969.96211


epoch 0
Loss: -16326.63671875
Loss: -17231.046875
Loss: -18022.953125
Train loss tensor(-16146.9766, grad_fn=<DivBackward1>)
Valid loss -17027.6640625
Accuracy 0.0
epoch 1
Loss: -16381.8095703125
Loss: -17289.333984375
Loss: -18084.4765625
Train loss tensor(-17423.5000, grad_fn=<DivBackward1>)
Valid loss -17085.352734375
Accuracy 0.0
epoch 2
Loss: -16437.095703125
Loss: -17347.427734375
Loss: -18145.857421875
Train loss tensor(-16843.6191, grad_fn=<DivBackward1>)
Valid loss -17142.980859375
Accuracy 0.0
epoch 3
Loss: -16492.345703125
Loss: -17405.5234375
Loss: -18207.23828125
Train loss tensor(-14436.7617, grad_fn=<DivBackward1>)
Valid loss -17200.5953125
Accuracy 0.0
epoch 4
Loss: -16547.81640625
Loss: -17464.19140625
Loss: -18268.953125
Train loss tensor(-15856.1309, grad_fn=<DivBackward1>)
Valid loss -17258.59375
Accuracy 0.0
epoch 5
Loss: -16603.625
Loss: -17523.23828125
Loss: -18330.953125
Train loss tensor(-18004.7617, grad_fn=<DivBackward1>)
Valid loss -17316.9359375
Accuracy 0.

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/train_loss,▄▃▄▇▃▅▄▆▅█▁▆█▃▄▃▅▇▁▃▂▅▇▄▆▅▄▃▆▂▅▃▂▄▆▅▄▄▂▄
val/val_accuracy,▁▁▁▁▁▁▁▁▁▁
val/val_loss,█▇▆▆▅▄▃▃▂▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,1050
train/train_loss,-17054.38086
val/val_accuracy,0.0
val/val_loss,-17553.04727


epoch 0
Loss: -17401.78515625
Loss: -18364.19140625
Loss: -19218.4765625
Train loss tensor(-19154.8574, grad_fn=<DivBackward1>)
Valid loss -18150.0859375
Accuracy 0.0
epoch 1
Loss: -17987.107421875
Loss: -18981.333984375
Loss: -19868.857421875
Train loss tensor(-17171.6191, grad_fn=<DivBackward1>)
Valid loss -18761.148046875
Accuracy 0.0
epoch 2
Loss: -18581.6015625
Loss: -19608.0
Loss: -20528.857421875
Train loss tensor(-21471.2383, grad_fn=<DivBackward1>)
Valid loss -19381.612109375
Accuracy 0.0
epoch 3
Loss: -19196.642578125
Loss: -20256.380859375
Loss: -21212.28515625
Train loss tensor(-19828.4766, grad_fn=<DivBackward1>)
Valid loss -20023.66640625
Accuracy 0.0
epoch 4
Loss: -19825.572265625
Loss: -20919.80859375
Loss: -21910.953125
Train loss tensor(-20403.8223, grad_fn=<DivBackward1>)
Valid loss -20680.34296875
Accuracy 0.0
epoch 5
Loss: -20471.322265625
Loss: -21600.19140625
Loss: -22627.427734375
Train loss tensor(-23131.0469, grad_fn=<DivBackward1>)
Valid loss -21354.091015625

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/train_loss,▇▇▇▆▆▆▆█▇▆▅▆▆▅▅▆▇▅▅▄▅▆▅▄▃▄▄▅▄▄▃▃▃▅▁▃▃▃▂▂
val/val_accuracy,▁▁▁▁▁▁▁▁▁▁
val/val_loss,█▇▇▆▅▄▄▃▂▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,1050
train/train_loss,-23823.57227
val/val_accuracy,0.0
val/val_loss,-24267.32852


epoch 0
Loss: -30896.69140625
Loss: -32597.333984375
Loss: -34256.3828125
Train loss tensor(-29238.8926, grad_fn=<DivBackward1>)
Valid loss -32248.88671875
Accuracy 0.0
epoch 1
Loss: -40151.2734375
Loss: -42356.953125
Loss: -44633.90625
Train loss tensor(-39994.0703, grad_fn=<DivBackward1>)
Valid loss -41930.071875
Accuracy 0.0
epoch 2
Loss: -51389.40625
Loss: -54196.5703125
Loss: -57262.85546875
Train loss tensor(-55694.4766, grad_fn=<DivBackward1>)
Valid loss -53686.96171875
Accuracy 0.0
epoch 3
Loss: -65020.90625
Loss: -68548.5703125
Loss: -72620.5703125
Train loss tensor(-64720.7617, grad_fn=<DivBackward1>)
Valid loss -67951.9046875
Accuracy 0.0
epoch 4
Loss: -81566.0
Loss: -85970.2890625
Loss: -91312.0
Train loss tensor(-81607.1406, grad_fn=<DivBackward1>)
Valid loss -85276.915625
Accuracy 0.0
epoch 5
Loss: -101401.5703125
Loss: -106857.140625
Loss: -113756.5703125
Train loss tensor(-101962.6641, grad_fn=<DivBackward1>)
Valid loss -106054.7984375
Accuracy 0.0
epoch 6
Loss: -125312

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/train_loss,████████▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▄▅▄▄▄▃▂▃▂▂▁
val/val_accuracy,▁▁▁▁▁▁▁▁▁▁
val/val_loss,██▇▇▆▆▅▄▂▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,1050
train/train_loss,-240214.09375
val/val_accuracy,0.0
val/val_loss,-239213.8625


epoch 0
Loss: -719868.9375
Loss: -757686.875
Loss: -817194.6875
Train loss tensor(-638698.6875, grad_fn=<DivBackward1>)
Valid loss -754461.2625
Accuracy 0.0
epoch 1
Loss: -1674206.5
Loss: -1761426.25
Loss: -1905712.75
Train loss tensor(-1623500.2500, grad_fn=<DivBackward1>)
Valid loss -1755395.65
Accuracy 0.0
epoch 2
Loss: -3314176.0
Loss: -3485208.5
Loss: -3778023.5
Train loss tensor(-2850621., grad_fn=<DivBackward1>)
Valid loss -3475358.5
Accuracy 0.0
epoch 3
Loss: -5938578.5
Loss: -6243474.5
Loss: -6775954.5
Train loss tensor(-5307538.5000, grad_fn=<DivBackward1>)
Valid loss -6228012.1
Accuracy 0.0
epoch 4
Loss: -9925723.0
Loss: -10434658.0
Loss: -11333925.0
Train loss tensor(-9182927., grad_fn=<DivBackward1>)
Valid loss -10410937.4
Accuracy 0.0
epoch 5
Loss: -15737100.0
Loss: -16542769.0
Loss: -17981244.0
Train loss tensor(-14258664., grad_fn=<DivBackward1>)
Valid loss -16508196.4
Accuracy 0.0
epoch 6
Loss: -23909742.0
Loss: -25133446.0
Loss: -27339434.0
Train loss tensor(-25929924

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/example_ct,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/train_loss,████████████████▇▇▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▃▃▃▂▂▁
val/val_accuracy,▁▁▁▁▁▁▁▁▁▁
val/val_loss,███▇▇▆▆▄▃▁
test_accuracy,0.8
train/epoch,10.0
train/example_ct,1050
train/train_loss,-69997520.0
val/val_accuracy,0.0
val/val_loss,-72871252.8
